In [45]:
import branca.colormap as cm  # Add this import

# Rest of your imports
import geopandas as gpd
from shapely.geometry import shape
import openrouteservice
from openrouteservice import exceptions
import pandas as pd
import folium
from folium.plugins import HeatMap

# Initialize ORS client and define generate_isochrones function
# (As shown in the previous section)

In [22]:
flats_with_pop=gpd.read_file('/home/silas/projects/msc_thesis/data/derived_data/flats_population.gpkg')
flats_with_pop.to_crs(epsg=4326, inplace=True)
rcps=gpd.read_file('/home/silas/projects/msc_thesis/data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')
rcps.to_crs(epsg=4326, inplace=True)

In [ ]:

# Initialize ORS client
api_key = os.getenv("ORS_API_KEY")
if not api_key:
    raise ValueError("OpenRouteService API key not found.")
client = openrouteservice.Client(key=api_key)

# Function to generate isochrones
def generate_isochrones(client, location, range_times):
    """
    Generate isochrones for a given location and range.

    Parameters:
    - client: ORS client instance.
    - location: [longitude, latitude].
    - range_times: list of time ranges in seconds.

    Returns:
    - GeoJSON of isochrones or None if failed.
    """
    try:
        params = {
            "locations": [location],
            "range": range_times,  # e.g., [60, 120, ..., 600] seconds
            "range_type": "time",
            "units": "s",  # Correct unit for 'time'
            "location_type": "start",
            "smoothing": 0.3,
            "profile": "foot-walking",
        }
        isochrones = client.isochrones(**params)
        return isochrones
    except exceptions.ApiError as e:
        print(f"API error for location {location}: {e}")
    except Exception as e:
        print(f"Unexpected error for location {location}: {e}")
    return None

# Example usage
# Assuming 'rcps' is your GeoDataFrame with recycling points
isochrones_list = []
for idx, row in rcps.iterrows():
    lon, lat = row.geometry.x, row.geometry.y
    time_ranges = list(range(60, 660, 60))  # [60, 120, ..., 600] seconds
    isochrones = generate_isochrones(client, [lon, lat], time_ranges)
    if isochrones:
        for feature in isochrones['features']:
            isochrones_list.append({
                'geometry': shape(feature['geometry']),
                'rcp_id': row['standort_i'],
                'time': feature['properties']['value']
            })
    else:
        print(f"Failed to generate isochrones for RCP ID {row['standort_i']}.")

# Convert to GeoDataFrame
isochrones_gdf = gpd.GeoDataFrame(isochrones_list, crs="EPSG:4326")

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered around Zurich
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Add isochrones to the map
for _, row in isochrones_gdf.iterrows():
    folium.GeoJson(row['geometry'], name=f"Isochrone {row['time']} min").add_to(m)

# Add recycling collection points to the map
marker_cluster = MarkerCluster().add_to(m)
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(marker_cluster)

# Display the map
m

In [ ]:
from shapely.ops import unary_union

def merge_isochrones_preserve_time(isochrones_gdf):
    """
    Merge isochrones preserving lower time values.

    Parameters:
    - isochrones_gdf: GeoDataFrame with isochrones and 'time' attribute.

    Returns:
    - GeoDataFrame with merged isochrones.
    """
    # Sort isochrones by 'time' ascending
    isochrones_sorted = isochrones_gdf.sort_values(by='time')

    merged_isochrones = gpd.GeoDataFrame(columns=isochrones_sorted.columns, crs="EPSG:4326")

    # Initialize an empty geometry for subtraction
    accumulated_geom = None

    for _, row in isochrones_sorted.iterrows():
        current_geom = row.geometry
        current_time = row['time']

        if accumulated_geom:
            remaining_geom = current_geom.difference(accumulated_geom)
        else:
            remaining_geom = current_geom

        if not remaining_geom.is_empty:
            new_row = row.copy()
            new_row.geometry = remaining_geom
            merged_isochrones = pd.concat([merged_isochrones, gpd.GeoDataFrame([new_row], crs="EPSG:4326")], ignore_index=True)
            # Update accumulated geometry
            if accumulated_geom:
                accumulated_geom = unary_union([accumulated_geom, remaining_geom])
            else:
                accumulated_geom = remaining_geom
        print(f"Processed isochrone with time {current_time} seconds.")

    return merged_isochrones

# Merge isochrones
merged_isochrones_gdf = merge_isochrones_preserve_time(isochrones_gdf)
print(merged_isochrones_gdf)

In [ ]:
# Function to prepare heatmap data

merged_isochrones_gdf.to_crs(epsg=4326, inplace=True)
def prepare_heatmap_data(merged_isochrones_gdf):
    """
    Prepare heatmap data from merged isochrones.

    Parameters:
    - merged_isochrones_gdf: GeoDataFrame with merged isochrones.

    Returns:
    - List of [latitude, longitude, intensity] for HeatMap.
    """
    heat_data = []
    for _, row in merged_isochrones_gdf.iterrows():
        if 'time' not in row:
            print("Missing 'time' column in merged_isochrones_gdf")
            continue
        centroid = row.geometry.centroid
        lat, lon = centroid.y, centroid.x
        intensity = row['time'] / 60  # Convert time to minutes for intensity
        heat_data.append([lat, lon, intensity])
    return heat_data
# Prepare data
heat_data = prepare_heatmap_data(merged_isochrones_gdf)

# Initialize Folium map
avg_lat = merged_isochrones_gdf.geometry.centroid.y.mean()
avg_lon = merged_isochrones_gdf.geometry.centroid.x.mean()
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

# Add HeatMap
HeatMap(heat_data, radius=11, blur=10, max_zoom=1).add_to(m)

# Add recycling collection points to the map
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(m)

# Create a color map
colormap = cm.LinearColormap(colors=['blue', 'green', 'yellow', 'orange', 'red'], vmin=0, vmax=10)
colormap.caption = 'Isochrone Time (minutes)'
colormap.add_to(m)

# Save and display the map
heatmap_path = '/home/silas/projects/msc_thesis/data/derived_data/heatmap_test.html'
m.save(heatmap_path)
m
